# Which csv?

## database.csv or fatalencounters??? or both??

We are going to try to see if we can use the name as a temporary key to mix the two databases

In [1]:
# Importing pandas

import pandas as pd
import datetime as dt

In [13]:
# Importing the 2 databases

wpost = pd.read_csv ('../data/database.csv')
fenc = pd.read_csv ('../data/fatalencounters.csv')

In [20]:
# Let´s have a quick look at the 2 databases.
# The Washington Post one:

wpost.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2142 entries, 0 to 2141
Data columns (total 14 columns):
id                         2142 non-null int64
name                       2123 non-null object
date                       2142 non-null object
manner_of_death            2142 non-null object
armed                      2136 non-null object
age                        2099 non-null float64
gender                     2141 non-null object
race                       2039 non-null object
city                       2142 non-null object
state                      2142 non-null object
signs_of_mental_illness    2142 non-null bool
threat_level               2142 non-null object
flee                       2106 non-null object
body_camera                2142 non-null bool
dtypes: bool(2), float64(1), int64(1), object(10)
memory usage: 205.1+ KB


In [21]:
# And now the Fatal encounters one:

fenc.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20792 entries, 0 to 20791
Data columns (total 24 columns):
Timestamp                                                                          20791 non-null float64
Subject's name                                                                     20792 non-null object
Subject's age                                                                      20207 non-null object
Subject's gender                                                                   20757 non-null object
Subject's race                                                                     20791 non-null object
URL of image of deceased                                                           7084 non-null object
Date of injury resulting in death (month/day/year)                                 20792 non-null object
Location of injury (address)                                                       20315 non-null object
Location of death (city)                           

At first glance it can appear the second one to be more in everything, more entries, more columns. The first thing is true: we have indeed more entries...but...more columns?

Name/ age / gender / race, are common to both databases ... so 4 - 4

talking about place: Wpost has city / state
                     Fenc has address/city/state/zip/county
                                                        ... so 6 - 9  (3 columns of additional info about location in fenc)
                                                        
We have date, manner (or cause) of death and mental illness in both..9 - 12

We have 4 unique columns in wpost:
Armed / Threat level / Flee / Body Camera                   ...13 - 12  ( 4 columns exclusive of wpost)

We have several unique columns in fenc, but some expendable:
URL of image of deceased / A brief description of the circumstances surrounding the death / Link to news article or photo of official document / State Data Status / Date&Description / Date (Year) / 20791 are expendable (we can keep the description for curiosity purposes)
We have 2 identifiers: de unique id and the timestamp ( we just need one)
We have 2 uniqe columns: agency and official dispotition    ...so 13 - 14  (2 uniques in fenc)


We have 5 columns unique in the fenc (3 of them about the location), and 4 unique in the wpost

In [22]:
# Comparing the 2 databases entries (just a remembrance)

wpost.shape, fenc.shape

((2142, 14), (20792, 24))

### Time frame of each database

Let´s see, more or less, wich is the difference of entries between our 2 databases considering a similar time frame.

In [23]:
fenc['Date (Year)'].min(), fenc['Date (Year)'].max()

(2000, 2100)

In [24]:
# Since we have a problem here, let´s see what is going to happen in the 2100


fenc[fenc['Date (Year)'] > 2017]

,Timestamp,Subject's name,Subject's age,Subject's gender,Subject's race,URL of image of deceased,Date of injury resulting in death (month/day/year),Location of injury (address),Location of death (city),Location of death (state),...,A brief description of the circumstances surrounding the death,Official disposition of death (justified or other),Link to news article or photo of official document,Symptoms of mental illness?,Unique identifier/submitted by,"State Data Status Prior to Jan. 1, 2013. All states complete after Jan. 1, 2013",Date&Description,20791,Unique identifier,Date (Year)
20791,NaN,Items below this row have not been fact-checked.,NaN,NaN,NaN,NaN,"December 31, 2100",Items below this line have not been fact-checked.,NaN,NaN,...,NaN,NaN,http://www.fatalencounters.org,NaN,Bill jones,X,12/31/2100: http://www.fatalencounters.org,20791.0,NaN,2100


In [25]:
# So, we have like and ending entrie that has no value to our database, le´s get rid of it

fenc = fenc.drop(20791)

In [26]:
fenc['Date (Year)'].min(), fenc['Date (Year)'].max()

(2000, 2017)

So our fatal encounters database goes from 2000 to 2017. We have solved the error of the 2100 entry dropping it.

In [27]:
# Let´s see the time frame of wpost, and the number of entries

wpost.date.min(), wpost.date.max(), wpost.id.count()

('2015-01-02', '2017-02-28', 2142)

In [28]:
# Approx, in the same time frame, in the fenc:

fenc[fenc['Date (Year)'] >= 2015].shape

(3919, 24)

We can see here that approximately, the fatal encounters db is doble the size of the washington post one. Let's try to be more accurate.

Since our column about the date of the killing in the fenc database is a string, and not a date, we cannot use it. We have to transform the data. In order to do that e are going to use the datetime library. We are going to use it to give format of a date to this column. We are going to create a new column where we are going to have this date data in a proper way.



In [30]:
# We iterate through every row, using iterrows. First we create our new column, with a string type

for i, row in fenc.iterrows():

    # we define n, as our string to be transformed for every entry
    n = row['Date of injury resulting in death (month/day/year)']

    # and here we define our new column, date, that is going to be n transformed into date. For doing that, we have to
    # describe which is the original format. In our case the date is written on a string like 'January 23, 2005'.
    # If we go to the documentation of the datetime library, we see that %B stands for the complete name of the month,
    # %d stands for the day number, then a coma, and the the year in 4 digits format.
    # The date in the end is to have just the date, without hours minutes and seconds
    fenc['date'].iloc[i] = dt.datetime.strptime(n,'%B %d, %Y')

C:\Users\ReDi\Anaconda3\lib\site-packages\pandas\core\indexing.py:141: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


In [33]:
fenc[(fenc['date'] >= '2015-01-02') & (fenc['date'] <= '2017-02-28')].shape

(3420, 25)

So in the end, for the same time frame the washington post dataframe is 2142 entries, the fatal encounters one is 3420. Can we try to merge them? Since the second one is 1300 entries bigger, we can asume that every name in the wpost is indeed in the fatal encounters one...but we can have typos, names that don´t match, missing names...let´s investigate.


## Merging the csv's

### Fixing the wpost names

Just for future uses, and because my computer is very slow, let's save this as a new csv

In [38]:
fenc.to_csv('fenc.csv', index = False, encoding='utf-8')

In [1]:
import pandas as pd
import datetime as dt

wpost = pd.read_csv ('../data/database.csv')
fenc = pd.read_csv ('../data/fenc.csv')

Just to avoid problems I copy everything that has to be charged again in here, to keep on with the analysis. I have uuploaded the new database, with the date added (fenc.csv).
The first thing to do will be be stablish the fenc database in the time frame of the washington post one. So we create the timefenc

In [2]:
timefenc = fenc[(fenc['date'] >= '2015-01-02') & (fenc['date'] <= '2017-02-28')]

As we said, it's probable that all the names of the wpost are included in the fenc, but, as we are going to use the name as a key, we need the wpost to be as complete as possible. Let's recall the sql analysis for the names from a few days ago. The brief result in it is as follows:

* Jamake Cason Thomas, duplicated. The entry is exactly the same. Delete one
* Daquan Antonio Westbrook, duplicated. Googling says the correct one is id 1129. We can get rid of id 2158.
* We have 19 null names
    * 1074 John Doe
    * 1570 John Doe...age 45, latin / correct shot for shot and tased
    * 1581 Tonnia Reshelle Jarvis, 48, white
    * 1584 Kevin T. Marshall, 30, white
    * 1615  John Doe...Latin
    * 1685 John Doe
    * 1848 John Doe...Latin, 35/40 age.../armed with rocks? previously tased?
    * 2037 Andrew Depeiza, 30 , Black, unarmed, not fleeing
    * 2110 John Doe, 60s
    * 2141 Walter Ricardo Pimentel, latin...correct 19th december 2016?
    * 2154 Luis Ambrosio-Aguilar, 30, latin
    * 2168 Jose Sanchez, 30 latin
    * 2164 Orande Kandie Hayes
    * 2182 Daniel Ralph Daily, 34 , white
    * 2232 David Darnell Stroughter, 50 , Black, Westchester (hood of LA?)
    * 2278 Solomon Picart, 37, Black
    * 2319 John Doe
    * 2328 John Doe
    * 2384 William Dwayne Darby, 39, White

In [3]:
wpost[wpost['name'] == 'Jamake Cason Thomas']

,id,name,date,manner_of_death,armed,age,gender,race,city,state,signs_of_mental_illness,threat_level,flee,body_camera
2049,2304,Jamake Cason Thomas,2017-02-03,shot,gun,22.0,M,B,Rowland,NC,False,attack,Car,False
2053,2294,Jamake Cason Thomas,2017-02-03,shot,gun,22.0,M,B,Rowland,NC,False,attack,Car,False


In [4]:
# First issue solved, easy

wpost = wpost.drop(2053)

In [5]:
wpost[wpost['name'] == 'Daquan Antonio Westbrook']

,id,name,date,manner_of_death,armed,age,gender,race,city,state,signs_of_mental_illness,threat_level,flee,body_camera
976,1129,Daquan Antonio Westbrook,2015-12-24,shot,gun,18.0,M,B,Charlotte,NC,False,attack,Not fleeing,False
1934,2158,Daquan Antonio Westbrook,2016-12-24,shot,gun,18.0,M,B,Charlotte,NC,False,undetermined,Not fleeing,False


In [6]:
# If we listen to our little list, we have to drop the second one. So:

wpost = wpost.drop(1934)

For the moment, we are going to leave the unknown people, or John Does, as they are right now. Let's change only the things we know.

In [ ]:
wpost[(wpost.id == 1074)+ 
    (wpost.id == 1570) + 
    (wpost.id == 1581) + 
    (wpost.id == 1584) + 
    (wpost.id == 1615) + 
    (wpost.id == 1685) + 
    (wpost.id == 1848) + 
    (wpost.id == 2037) + 
    (wpost.id == 2110) + 
    (wpost.id == 2141) + 
    (wpost.id == 2154) + 
    (wpost.id == 2168) + 
    (wpost.id == 2164) + 
    (wpost.id == 2182) + 
    (wpost.id == 2232) + 
    (wpost.id == 2278) + 
    (wpost.id == 2319) + 
    (wpost.id == 2328) + 
    (wpost.id == 2384) 
]

In [23]:
# (928) 1074 John Doe
        # nothing to do
    
# (1382) 1570 John Doe...age 45, latin (correct shot for shot and tased, in hold)
wpost['age'].iloc[1382] = 45
wpost['race'].iloc[1382] = 'H'
wpost[(wpost.id == 1074)+ 
    (wpost.id == 1570)]

C:\Users\ReDi\Anaconda3\lib\site-packages\pandas\core\indexing.py:141: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)
C:\Users\ReDi\Anaconda3\lib\site-packages\pandas\computation\expressions.py:182: UserWarning: evaluating in Python space because the '+' operator is not supported by numexpr for the bool dtype, use '|' instead
  unsupported[op_str]))


,id,name,date,manner_of_death,armed,age,gender,race,city,state,signs_of_mental_illness,threat_level,flee,body_camera
928,1074,NaN,2015-12-10,shot,gun,NaN,M,NaN,Hemet,CA,False,attack,Not fleeing,False
1382,1570,NaN,2016-05-26,shot,gun,45.0,M,H,San Antonio,TX,False,other,Not fleeing,False


In [25]:
# (1385) 1581 Tonnia Reshelle Jarvis, 48, white
wpost['name'].iloc[1385] = 'Tonnia Reshelle Jarvis'
wpost['age'].iloc[1385] = 48
wpost['race'].iloc[1385] = 'W'
wpost[(wpost.id == 1581)]

C:\Users\ReDi\Anaconda3\lib\site-packages\pandas\core\indexing.py:141: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


,id,name,date,manner_of_death,armed,age,gender,race,city,state,signs_of_mental_illness,threat_level,flee,body_camera
1385,1581,Tonnia Reshelle Jarvis,2016-05-26,shot,knife,48.0,F,W,Sneads,FL,False,other,Not fleeing,False


In [26]:
# (1397) 1584 Kevin T. Marshall, 30, white
wpost['name'].iloc[1397] = 'Kevin T. Marshall'
wpost['age'].iloc[1397] = 30
wpost['race'].iloc[1397] = 'W'
wpost[(wpost.id == 1584)]

C:\Users\ReDi\Anaconda3\lib\site-packages\pandas\core\indexing.py:141: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


,id,name,date,manner_of_death,armed,age,gender,race,city,state,signs_of_mental_illness,threat_level,flee,body_camera
1397,1584,Kevin T. Marshall,2016-06-01,shot,gun,30.0,M,W,Phoenix,AZ,False,attack,Not fleeing,False


In [27]:
# (1419) 1615  John Doe...Latin
wpost['race'].iloc[1419] = 'H'
wpost[(wpost.id == 1615)]
# (1500) 1685 John Doe
    #nothing to do

C:\Users\ReDi\Anaconda3\lib\site-packages\pandas\core\indexing.py:141: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


,id,name,date,manner_of_death,armed,age,gender,race,city,state,signs_of_mental_illness,threat_level,flee,body_camera
1419,1615,NaN,2016-06-09,shot,baton,NaN,M,H,Somerton,AZ,False,attack,Not fleeing,False


In [29]:
# (1646) 1848 John Doe...Latin, 35/40...38 rounding age.../armed with rocks? previously tased?
wpost['age'].iloc[1646] = 38
wpost['race'].iloc[1646] = 'H'
wpost[(wpost.id == 1848)]

C:\Users\ReDi\Anaconda3\lib\site-packages\pandas\core\indexing.py:141: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


,id,name,date,manner_of_death,armed,age,gender,race,city,state,signs_of_mental_illness,threat_level,flee,body_camera
1646,1848,NaN,2016-09-01,shot and Tasered,pipe,38.0,M,H,Huntington Park,CA,False,other,Not fleeing,False


In [32]:
# (1825) 2037 Andrew Depeiza, 30 , Black, unarmed, not fleeing
wpost['name'].iloc[1825] = 'Andrew Depeiza'
wpost['age'].iloc[1825] = 30
wpost['race'].iloc[1825] = 'B'
wpost['armed'].iloc[1825] = 'unarmed'
wpost['flee'].iloc[1825] = 'Not fleeing'
wpost[(wpost.id == 2037)]

C:\Users\ReDi\Anaconda3\lib\site-packages\pandas\core\indexing.py:141: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


,id,name,date,manner_of_death,armed,age,gender,race,city,state,signs_of_mental_illness,threat_level,flee,body_camera
1825,2037,Andrew Depeiza,2016-11-11,shot and Tasered,unarmed,30.0,M,B,East Point,GA,False,undetermined,Not fleeing,False


In [33]:
# (1894) 2110 John Doe, 60s...60
wpost['age'].iloc[1894] = 60
wpost[(wpost.id == 2110)]

C:\Users\ReDi\Anaconda3\lib\site-packages\pandas\core\indexing.py:141: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


,id,name,date,manner_of_death,armed,age,gender,race,city,state,signs_of_mental_illness,threat_level,flee,body_camera
1894,2110,NaN,2016-12-08,shot,gun,60.0,M,NaN,Allen,TX,False,attack,Not fleeing,False


In [35]:
# (1921) 2141 Walter Ricardo Pimentel, latin...correct 19th december 2016? (not yet)
wpost['name'].iloc[1921] = 'Walter Ricardo Pimentel'
wpost['race'].iloc[1921] = 'H'
wpost[(wpost.id == 2141)]

C:\Users\ReDi\Anaconda3\lib\site-packages\pandas\core\indexing.py:141: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


,id,name,date,manner_of_death,armed,age,gender,race,city,state,signs_of_mental_illness,threat_level,flee,body_camera
1921,2141,Walter Ricardo Pimentel,2016-12-20,shot,knife,NaN,M,H,Brawley,CA,True,other,Not fleeing,False


In [36]:
# (1927) 2154 Luis Ambrosio-Aguilar, 30, latin
wpost['name'].iloc[1927] = 'Luis Ambrosio-Aguilar'
wpost['age'].iloc[1927] = 30
wpost['race'].iloc[1927] = 'H'
wpost[(wpost.id == 2154)]

C:\Users\ReDi\Anaconda3\lib\site-packages\pandas\core\indexing.py:141: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


,id,name,date,manner_of_death,armed,age,gender,race,city,state,signs_of_mental_illness,threat_level,flee,body_camera
1927,2154,Luis Ambrosio-Aguilar,2016-12-21,shot,knife,30.0,M,H,Stockton,CA,False,undetermined,Not fleeing,False


In [37]:
# (1933) 2168 Jose Sanchez, 30 latin
wpost['name'].iloc[1933] = 'Jose Sanchez'
wpost['age'].iloc[1933] = 30
wpost['race'].iloc[1933] = 'H'
wpost[(wpost.id == 2168)]

C:\Users\ReDi\Anaconda3\lib\site-packages\pandas\core\indexing.py:141: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


,id,name,date,manner_of_death,armed,age,gender,race,city,state,signs_of_mental_illness,threat_level,flee,body_camera
1933,2168,Jose Sanchez,2016-12-23,shot,vehicle,30.0,M,H,El Monte,CA,False,other,Car,False


In [43]:
# (1936) 2164 Orande Kandie Hayes
wpost.loc[wpost['id'] == 2164, "name"] = "Orande Kandie Hayes"
wpost[(wpost.id == 2164)]

,id,name,date,manner_of_death,armed,age,gender,race,city,state,signs_of_mental_illness,threat_level,flee,body_camera
1936,2164,Orande Kandie Hayes,2016-12-24,shot,gun,NaN,M,NaN,Gadsden,AL,False,attack,Foot,False


In [44]:
# (1949) 2182 Daniel Ralph Daily, 34 , white
wpost.loc[wpost['id'] == 2182, "name"] = "Daniel Ralph Daily"
wpost.loc[wpost['id'] == 2182, "age"] = 34
wpost.loc[wpost['id'] == 2182, "race"] = "W"
wpost[(wpost.id == 2182)]

,id,name,date,manner_of_death,armed,age,gender,race,city,state,signs_of_mental_illness,threat_level,flee,body_camera
1949,2182,Daniel Ralph Daily,2016-12-30,shot,gun,34.0,M,W,Pensacola,FL,False,attack,Not fleeing,False


In [45]:
# (2003) 2232 David Darnell Stroughter, 50 , Black, Westchester (hood of LA?)
wpost.loc[wpost['id'] == 2232, "name"] = "David Darnell Stroughter"
wpost.loc[wpost['id'] == 2232, "age"] = 50
wpost.loc[wpost['id'] == 2232, "race"] = "B"
wpost[(wpost.id == 2232)]

,id,name,date,manner_of_death,armed,age,gender,race,city,state,signs_of_mental_illness,threat_level,flee,body_camera
2003,2232,David Darnell Stroughter,2017-01-18,shot,hatchet,50.0,M,B,Los Angeles,CA,False,other,Car,False


In [46]:
# (2044) 2278 Solomon Picart, 37, Black
wpost.loc[wpost['id'] == 2278, "name"] = "Solomon Picart"
wpost.loc[wpost['id'] == 2278, "age"] = 37
wpost.loc[wpost['id'] == 2278, "race"] = "B"
wpost[(wpost.id == 2278)]

# (2077) 2319 John Doe
    #Nothing to do
# (2090) 2328 John Doe
    #Notning to do

,id,name,date,manner_of_death,armed,age,gender,race,city,state,signs_of_mental_illness,threat_level,flee,body_camera
2044,2278,Solomon Picart,2017-01-31,shot and Tasered,knife,37.0,M,B,Hollywood,CA,True,attack,Not fleeing,False


In [47]:
# (2133) 2384 William Dwayne Darby, 39, White
wpost.loc[wpost['id'] == 2384, "name"] = "William Dwayne Darby"
wpost.loc[wpost['id'] == 2384, "age"] = 39
wpost.loc[wpost['id'] == 2384, "race"] = "W"
wpost[(wpost.id == 2384)]

,id,name,date,manner_of_death,armed,age,gender,race,city,state,signs_of_mental_illness,threat_level,flee,body_camera
2133,2384,William Dwayne Darby,2017-02-25,shot,gun,39.0,NaN,W,Lumpkin,GA,False,attack,Car,False


In [48]:
wpost[(wpost.id == 1074)+ 
    (wpost.id == 1570) + 
    (wpost.id == 1581) + 
    (wpost.id == 1584) + 
    (wpost.id == 1615) + 
    (wpost.id == 1685) + 
    (wpost.id == 1848) + 
    (wpost.id == 2037) + 
    (wpost.id == 2110) + 
    (wpost.id == 2141) + 
    (wpost.id == 2154) + 
    (wpost.id == 2168) + 
    (wpost.id == 2164) + 
    (wpost.id == 2182) + 
    (wpost.id == 2232) + 
    (wpost.id == 2278) + 
    (wpost.id == 2319) + 
    (wpost.id == 2328) + 
    (wpost.id == 2384) 
]

C:\Users\ReDi\Anaconda3\lib\site-packages\pandas\computation\expressions.py:182: UserWarning: evaluating in Python space because the '+' operator is not supported by numexpr for the bool dtype, use '|' instead
  unsupported[op_str]))


,id,name,date,manner_of_death,armed,age,gender,race,city,state,signs_of_mental_illness,threat_level,flee,body_camera
928,1074,NaN,2015-12-10,shot,gun,NaN,M,NaN,Hemet,CA,False,attack,Not fleeing,False
1382,1570,NaN,2016-05-26,shot,gun,45.0,M,H,San Antonio,TX,False,other,Not fleeing,False
1385,1581,Tonnia Reshelle Jarvis,2016-05-26,shot,knife,48.0,F,W,Sneads,FL,False,other,Not fleeing,False
1397,1584,Kevin T. Marshall,2016-06-01,shot,gun,30.0,M,W,Phoenix,AZ,False,attack,Not fleeing,False
1419,1615,NaN,2016-06-09,shot,baton,NaN,M,H,Somerton,AZ,False,attack,Not fleeing,False
1500,1685,NaN,2016-07-04,shot,knife,NaN,M,NaN,Rosser,TX,False,other,Not fleeing,True
1646,1848,NaN,2016-09-01,shot and Tasered,pipe,38.0,M,H,Huntington Park,CA,False,other,Not fleeing,False
1825,2037,Andrew Depeiza,2016-11-11,shot and Tasered,unarmed,30.0,M,B,East Point,GA,False,undetermined,Not fleeing,False
1894,2110,NaN,2016-12-08,shot,gun,60.0,M,NaN,Allen,TX,False,attack,Not fleeing,False
1921,2141,Walter Ricardo Pimentel,2016-12-20,shot,knife,NaN,M,H,Brawley,CA,True,other,Not fleeing,False


This is the best our names from the wpost can be (together with some data of this people). So the best thing to do is to save the new csv, just in case

In [49]:
wpost.to_csv('wpost.csv', index = False, encoding='utf-8')

### Name is the key

In [52]:
# Again, for security purposes, I redifine and upload everything necessary

import pandas as pd
import datetime as dt

wpost = pd.read_csv ('../data/wpost.csv')
fenc = pd.read_csv ('../data/fenc.csv')
timefenc = fenc[(fenc['date'] >= '2015-01-02') & (fenc['date'] <= '2017-02-28')]

Let´s define a dictionary of timefenc, with name as key, date as value. Maybe it will help us to see if we can merge this

In [125]:
names_list = []

for i, row in timefenc.iterrows():

    # we define names and dates
    name = row[1]
    date = row [24]
    tuplet = (name,date)
    
    #and we add our data to a list
    names_list.append (tuplet)

In [126]:
# and we transform our list to a dictionary
names_fenc = {}
    
for name,date in names_list:
    names_fenc[name] = date

Now we ask to our wonderful dictionary how many people do we have in it

In [127]:
n = 0

for i, row in wpost.iterrows():

    #we create our counter and our name and date
    
    name = row[1]
    date = row [2]
    
    # and now, the question
    if (name in names_fenc.keys())&(date in names_fenc.values()):
        n = n+1
print (n)

1437


This is promising. We have 1437 of the wpost dataframe with an equal entry in name and date in fenc dataframe. Lets check just the name.

In [128]:
n = 0

for i, row in wpost.iterrows():

    #we create our counter and our name
    
    name = row[1]
    
    # and now, the question
    if (name in names_fenc.keys()):
        n = n+1
print (n)

1440


This does not throw more light. Let´s check first what happens with the 1437.

In [129]:
commonnames= []

for i, row in wpost.iterrows():

    # we define names and dates
    name = row[1]
    date = row [2]
    
    #our question to add people to our list is...
    if (name in names_fenc.keys())&(date in names_fenc.values()):
        commonnames.append (name)

In [130]:
commonpost = wpost

In [131]:
commonpost = commonpost[commonpost['name'].isin(commonnames)]

In [132]:
commonpost

,id,name,date,manner_of_death,armed,age,gender,race,city,state,signs_of_mental_illness,threat_level,flee,body_camera
1,4,Lewis Lee Lembke,2015-01-02,shot,gun,47.0,M,W,Aloha,OR,False,attack,Not fleeing,False
2,5,John Paul Quintero,2015-01-03,shot and Tasered,unarmed,23.0,M,H,Wichita,KS,False,other,Not fleeing,False
3,8,Matthew Hoffman,2015-01-04,shot,toy weapon,32.0,M,W,San Francisco,CA,True,attack,Not fleeing,False
6,13,Kenneth Arnold Buck,2015-01-05,shot,gun,22.0,M,H,Chandler,AZ,False,attack,Car,False
7,15,Brock Nichols,2015-01-06,shot,gun,35.0,M,W,Assaria,KS,False,attack,Not fleeing,False
9,17,Leslie Sapp III,2015-01-06,shot,toy weapon,47.0,M,B,Knoxville,PA,False,attack,Not fleeing,False
12,22,Hashim Hanif Ibn Abdul-Rasheed,2015-01-07,shot,knife,41.0,M,B,Columbus,OH,True,other,Not fleeing,False
13,25,Nicholas Ryan Brickman,2015-01-07,shot,gun,30.0,M,W,Des Moines,IA,False,attack,Car,False
14,27,Omarr Julian Maximillian Jackson,2015-01-07,shot,gun,37.0,M,B,New Orleans,LA,False,attack,Foot,True
16,32,James Dudley Barker,2015-01-08,shot,shovel,42.0,M,W,Salt Lake City,UT,False,attack,Not fleeing,True


In [133]:
commonfenc = timefenc

In [134]:
commonfenc = commonfenc[commonfenc["Subject's name"].isin(commonnames)]

In [135]:
commonfenc

,Timestamp,Subject's name,Subject's age,Subject's gender,Subject's race,URL of image of deceased,Date of injury resulting in death (month/day/year),Location of injury (address),Location of death (city),Location of death (state),...,Official disposition of death (justified or other),Link to news article or photo of official document,Symptoms of mental illness?,Unique identifier/submitted by,"State Data Status Prior to Jan. 1, 2013. All states complete after Jan. 1, 2013",Date&Description,20791,Unique identifier,Date (Year),date
16878,15580.0,Lewis Lee Lembke,47,Male,Race unspecified,NaN,"January 2, 2015",4519 S.W. Masters Loop,Aloha,OR,...,Unreported,http://www.oregonlive.com/aloha/index.ssf/2015...,Drug or alcohol use,PThompson,Complete,1/2/2015: Deputies responded to a call from th...,NaN,15580.0,2015.0,2015-01-02
16881,15583.0,John Paul Quintero,23,Male,Hispanic/Latino,https://lintvksnw.files.wordpress.com/2015/01/...,"January 3, 2015",500 N Oliver Ave,Wichita,KS,...,Unreported,http://www.kwch.com/news/local-news/officerinv...,No,KBP/Irish,Complete,1/3/2015: The 23-year-old was shot twice by a ...,NaN,15583.0,2015.0,2015-01-03
16883,15586.0,Matthew Hoffman,32,Male,European-American/White,NaN,"January 4, 2015",Valencia Street and 17th Street,San Francisco,CA,...,Pending investigation,http://www.insidebayarea.com/breaking-news/ci_...,Unknown,gunviolencearchive.org/Irish,Complete,1/4/2015: A man shot and killed by San Francis...,NaN,15586.0,2015.0,2015-01-04
16887,15587.0,Kenneth Arnold Buck,22,Male,Native American/Alaskan,http://www.gannett-cdn.com/-mm-/3221a1f017874e...,"January 5, 2015",E Knox Rd,Chandler,AZ,...,Unreported,http://www.azcentral.com/story/news/local/chan...,No,gunviolencearchive.org/Irish,Complete,1/5/2015: Buck was driving westbound on Knox R...,NaN,15587.0,2015.0,2015-01-05
16891,15591.0,Brock Nichols,35,Male,European-American/White,http://d1t3gia0in9tdj.cloudfront.net/photo/tri...,"January 6, 2015",2388 E. Kansas Highway 4,Assaria,KS,...,Justified,http://ksnt.com/2015/01/07/kansas-man-dies-aft...,Drug or alcohol use,kbp/AlexV,Complete,1/6/2015: Officers responded to a call from a ...,NaN,15591.0,2015.0,2015-01-06
16892,15590.0,Leslie Sapp III,47,Male,African-American/Black,https://cbspittsburgh.files.wordpress.com/2015...,"January 6, 2015",101 Redlyn Street,Knoxville,PA,...,Unreported,http://www.post-gazette.com/local/city/2015/01...,No,PThompson,Complete,1/6/2015: Marshals went to serve a rape warran...,NaN,15590.0,2015.0,2015-01-06
16893,15594.0,Nicholas Ryan Brickman,30,Male,European-American/White,https://localtvwhotv.files.wordpress.com/2015/...,"January 7, 2015",SE 4th St. and Scott Ave.,Des Moines,IA,...,Pending investigation,http://whotv.com/2015/01/07/suspect-shot-follo...,No,kbp/AlexV,Complete,1/7/2015: Officers were called to a robbery at...,NaN,15594.0,2015.0,2015-01-07
16895,15595.0,Omarr Julian Maximillian Jackson,37,Male,African-American/Black,NaN,"January 7, 2015",Lasalle St & Josephine St,New Orleans,LA,...,Justified,http://www.wwltv.com/story/news/crime/2015/01/...,No,erin.griebel@gmail.com,Complete,1/7/2015: Officer Matthew Bencik and his partn...,NaN,15595.0,2015.0,2015-01-07
16897,15596.0,Hashim Hanif Ibn Abdul-Rasheed,41,Male,African-American/Black,http://www.killedbypolice.net/victims/150016.jpg,"January 7, 2015",4600 International Gateway,Columbus,OH,...,Justified,http://www.nydailynews.com/news/crime/man-arme...,Unknown,New/PThompson,Complete,1/7/2015: Police were called when the illegall...,NaN,15596.0,2015.0,2015-01-07
16900,15600.0,Artago Damon Howard,36,Male,African-American/Black,http://thumbs.mugshots.com/gallery/images/98/e...,"January 8, 2015",9661 Strong Highway,Strong,AR,...,Pending investigation,http://www.greenfieldreporter.com/view/story/f...,No,New/PThompson,Complete,1/8/2015: Police responded to a burglar alrm a...,NaN,15600.0,2015.0,2015-01-08


Is it possible we have a repeated entry? In fact this has to remember us that we have to clean as well this dataframe...having that in mind, let's see what happens in our time framed data frame

In [112]:
len(commonfenc["Subject's name"].unique())

1433

In [118]:
# Sorry, but I could't make it nicer...let´s check what happens with this five guys

commonfenc["Subject's name"].value_counts() > 1

Michael Johnson                True
Brandon Jones                  True
Antonio Perez                  True
Antonio Gonzalez               True
James Brown III                True
Thomas Hirko                  False
Terry Salazar                 False
Jorge Brian Gonzalez          False
Angel Ramos                   False
David L. Anderson             False
Darius Wimberly               False
Timothy Milliken              False
Jose Perez                    False
Joseph Harris                 False
Alexander Chance Partain      False
Juan Gabriel Torres           False
Tian Ma                       False
Daniel Scott Giberson         False
Eddie Collins                 False
Mark Anthony Hicks            False
Douglas Yon                   False
John O'Handley                False
Pablo Renato Cartagena        False
Michael Eugene Wilson Jr.     False
Shane Watkins                 False
Francis Clark                 False
Bobby Daniels                 False
Quincy Reed Reindl          

In [124]:
# They are pretty common name, except our suspiciuous James Brown the third...
suspects = ['Michael Johnson','Brandon Jones','Antonio Perez','Antonio Gonzalez','James Brown III']
commonfenc[commonfenc["Subject's name"].isin(suspects)].sort(["Subject's name"])   

C:\Users\ReDi\Anaconda3\lib\site-packages\ipykernel\__main__.py:3: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
  app.launch_new_instance()


,Timestamp,Subject's name,Subject's age,Subject's gender,Subject's race,URL of image of deceased,Date of injury resulting in death (month/day/year),Location of injury (address),Location of death (city),Location of death (state),...,Official disposition of death (justified or other),Link to news article or photo of official document,Symptoms of mental illness?,Unique identifier/submitted by,"State Data Status Prior to Jan. 1, 2013. All states complete after Jan. 1, 2013",Date&Description,20791,Unique identifier,Date (Year),date
17696,16380.0,Antonio Gonzalez,29,Male,Hispanic/Latino,https://localtvwiti.files.wordpress.com/2015/0...,"July 16, 2015",8500 Glencoe Cir,Wauwatosa,WI,...,Unreported,http://fox6now.com/2015/07/17/friend-of-wauwat...,Unknown,Guardian/,Complete,"7/16/2015: Police were called when Gonzalez, w...",NaN,16380.0,2015.0,2015-07-16
18927,17591.0,Antonio Gonzalez,34,Male,Hispanic/Latino,NaN,"April 26, 2016",18 Melrose St,Lawrence,MA,...,Suicide,http://boston.cbslocal.com/2016/04/26/lawrence...,No,NaN,Complete,4/26/2016: Gonzalez barricaded himself in his ...,NaN,17591.0,2016.0,2016-04-26
17139,15836.0,Antonio Perez,32,Male,Hispanic/Latino,https://2dbdd5116ffa30a49aa8-c03f075f8191fb4e6...,"March 12, 2015",2400 block Flower Street,Los Angeles,CA,...,Unreported,http://www.presstelegram.com/general-news/2015...,Unknown,kbp/PThompson,Complete,3/12/2015: The subject was allegedly driving a...,NaN,15836.0,2015.0,2015-03-12
17452,16138.0,Antonio Perez,15,Male,Race unspecified,NaN,"May 21, 2015",Stuart Rd & Ladd Rd,Corry,PA,...,Unreported,http://www.goerie.com/police-find-cache-of-gun...,No,Walt Lockley,Complete,5/21/2015: Five suspects fleeing from a burgla...,NaN,16138.0,2015.0,2015-05-21
17174,15868.0,Brandon Jones,18,Male,African-American/Black,NaN,"March 19, 2015",1077 Parkwood Dr,Cleveland,OH,...,Unreported,http://newsone.com/3100355/brandon-jones-unarm...,Unknown,New JoshN,Complete,3/19/2015: Officers reported that they respond...,NaN,15868.0,2015.0,2015-03-19
19689,18347.0,Brandon Jones,25,Male,European-American/White,NaN,"October 25, 2016",291 E Pulaski Hwy,Elkton,MD,...,Pending investigation,http://www.cecildaily.com/spotlight/article_03...,No,NaN,Incomplete,10/25/2016: Brandon Jones and Chelsea M. Porte...,NaN,18347.0,2016.0,2016-10-25
17902,16582.0,James Brown III,25,Male,African-American/Black,http://1.bp.blogspot.com/-xOS3t4XLRDc/VegsAfvO...,"August 29, 2015",Losee Rd and E Sharp Cir,North Las Vegas,NV,...,Pending investigation,http://www.reviewjournal.com/news/las-vegas/ma...,No,SLS,Complete,8/29/2015: Brown was shot during when he turne...,NaN,16582.0,2015.0,2015-08-29
18833,17492.0,James Brown III,34,Male,African-American/Black,http://i.dailymail.co.uk/i/pix/2016/04/01/20/3...,"March 31, 2016",2910 N Boulevard,Richmond,VA,...,Pending investigation,http://www.dailymail.co.uk/news/article-351870...,No,PTC,Incomplete,3/31/2016: Subject with an alleged history of ...,NaN,17492.0,2016.0,2016-03-31
18201,16867.0,Michael Johnson,51,Male,European-American/White,http://media.katu.com/images/151109_Michael_Jo...,"November 6, 2015",1015 NW 22nd Ave,Portland,OR,...,Pending investigation,http://www.oregonlive.com/portland/index.ssf/2...,Yes,Guardian/Nick,Complete,11/6/2015: Johnson was holding a gun to his he...,NaN,16867.0,2015.0,2015-11-06
19071,17731.0,Michael Johnson,21,Male,African-American/Black,http://www.fatalencounters.org/wp-content/uplo...,"June 1, 2016",590 Bullsboro Dr,Newnan,GA,...,Pending investigation,http://www.ajc.com/news/news/crime-law/man-kil...,No,NaN,Complete,6/1/2016: Johnson fled on foot from a traffic ...,NaN,17731.0,2016.0,2016-06-01
